<a href="https://colab.research.google.com/github/mralamdari/YOLO/blob/main/Yolo_v8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
from IPython.display import clear_output

In [2]:
#install necessary libraries
!pip install awscli
!pip install ultralytics
clear_output()

#Data

In [3]:
#clone OIDv4_ToolKit
!git clone https://github.com/mralamdari/OIDv4_ToolKit.git

Cloning into 'OIDv4_ToolKit'...
remote: Enumerating objects: 499, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 499 (delta 32), reused 52 (delta 31), pack-reused 444
Receiving objects: 100% (499/499), 34.11 MiB | 18.74 MiB/s, done.
Resolving deltas: 100% (189/189), done.


In [4]:
%cd OIDv4_ToolKit

#Write your desired objects (--classes) and data type (--type_csv)  and the number of images you want (--limit)
# !python3 main.py downloader --classes Truck --type_csv train --limit 100

!python3 main.py downloader --classes Truck Person Tree --type_csv train --limit 50

/content/OIDv4_ToolKit

		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Truck.
   [ERROR] | Missing the class-descriptions-boxable.csv file.
[DOWNLOAD] | Do you want to download the missing file? [Y/n] Y
...145%, 0 MB, 34788 KB/s, 0 seconds pass

In [5]:
!python mralamdari_annotations_converter.py

Currently in subdirectory: train
Converting annotations for class: Tree
100% 50/50 [00:05<00:00,  9.93it/s]
Converting annotations for class: Person
100% 50/50 [00:02<00:00, 22.52it/s]
Converting annotations for class: Truck
100% 50/50 [00:01<00:00, 45.39it/s]


In [10]:
path = '/content/data/obj/train/Truck/'
os.makedirs('/content/train/images', exist_ok=True)
os.makedirs('/content/train/labels', exist_ok=True)
# os.rename('/content/data', '/content/old_data')
# os.rename('/content/train', '/content/data')


for i in os.listdir(path):
  file_type = 'labels' if i[-4:] == '.txt' else 'images'
  os.rename(path+i, f'/content/train/{file_type}/{i}')

In [8]:
os.rename('/content/data', '/content/old_data')
os.rename('/content/train', '/content/data')

In [ ]:
# !pip install ultralytics
from ultralytics import YOLO
model = YOLO('yolov8n.yaml')

In [17]:
DATA_ROOT = '/content/data/'

with open('/content/config.yaml', 'w+') as f:
    config_files = f.write(f"""path: '{DATA_ROOT}' # dataset root dir
train: images  # train images (relative to 'path')
val: val_images  # val images (relative to 'path')

# Classes
names:
  0: Truck""")


results = model.train(data='/content/config.yaml', epochs=1)

Ultralytics YOLOv8.0.149 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/config.yaml, epochs=1, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=0, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simp

RuntimeError: ignored

#Detection

In [ ]:
# path = '/content/data/obj/train/'
# os.makedirs('/content/train/images', exist_ok=True)
# os.makedirs('/content/train/labels', exist_ok=True)

# for j in os.listdir('/content/data/obj/train/'):
#   for i in os.listdir(path+j):
#     file_type = 'labels' if i[-4:] == '.txt' else 'images'
#     os.rename(path+j+'/'+i, f'/content/train/{file_type}/{i}')




with open('/content/config.yaml', 'w+') as f:
    config_files = f.write(f"""path: /content/train # dataset root dir
train: images  # train images (relative to 'path')
val: images  # val images (relative to 'path')


# Classes
names:
  0: Truck
  1: Person
  2: Tree""")

results = model.train(data='/content/config.yaml', epochs=10)

Ultralytics YOLOv8.0.149 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/config.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simpl

In [7]:
# os.makedirs('/content/data/obj/train/Labels/', exist_ok=True)
# for i in os.listdir('/content/data/obj/train/Truck/'):
#   if i[-4:] == '.txt':
#     os.rename(f'/content/data/obj/train/Truck/{i}', f'/content/data/obj/train/Labels/{i}')

In [6]:
# os.makedirs(f'/content/data/images/train/', exist_ok=True)
# os.makedirs(f'/content/data/labels/train/', exist_ok=True)
# os.makedirs(f'/content/data/images/test/', exist_ok=True)
# os.makedirs(f'/content/data/labels/test/', exist_ok=True)

# path = '/content/data/obj/train/'
# DATA_ROOT = '/content/data/'

# VAL_THRESHOLD = 0.2

# for category in os.listdir(path):
#   category_path = path+category

#   for i in os.listdir(category_path):
#     if i[-4:] == '.txt':
#       file_name = i[:-4]
#       image_file = category_path + '/' + file_name + '.jpg'
#       label_file = category_path + '/' + file_name + '.txt'
#       rand_num = np.random.rand()
#       target = 'test' if rand_num <= VAL_THRESHOLD else 'train'
#       os.rename(image_file, f'/content/data/images/{target}/{file_name}.img')
#       os.rename(label_file, f'/content/data/labels/{target}/{file_name}.tx')

In [6]:
os.makedirs(f'/content/data/train/images/', exist_ok=True)
os.makedirs(f'/content/data/train/labels/', exist_ok=True)
os.makedirs(f'/content/data/test/images/', exist_ok=True)
os.makedirs(f'/content/data/test/labels/', exist_ok=True)


path = '/content/data/obj/train/'
DATA_ROOT = '/content/data/'

VAL_THRESHOLD = 0.2

for category in os.listdir(path):
  category_path = path+category

  for i in os.listdir(category_path):
    if i[-4:] == '.txt':
      file_name = i[:-4]
      image_file = category_path + '/' + file_name + '.jpg'
      label_file = category_path + '/' + file_name + '.txt'
      rand_num = np.random.rand()
      target = 'test' if rand_num <= VAL_THRESHOLD else 'train'
      os.rename(image_file, f'/content/data/{target}/images/{file_name}.img')
      os.rename(label_file, f'/content/data/{target}/labels/{file_name}.tx')

In [7]:
os.rename('/content/data/obj', '/content/obj')
os.rename('/content/data/obj.names', '/content/obj.names')
os.rename('/content/data/train.txt', '/content/train.txt')

In [11]:
from ultralytics import YOLO

# model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt')

100%|██████████| 6.23M/6.23M [00:00<00:00, 49.3MB/s]


In [14]:
# with open('/content/config.yaml', 'w+') as f:
#     config_files = f.write(f"""path: /content/data/images/ # dataset root dir
# train: train  # train images (relative to 'path')
# val: test  # val images (relative to 'path')

# # Classes
# names:
#   0: Truck""")

In [13]:
with open('/content/config.yaml', 'w+') as f:
    config_files = f.write(f"""path: /content/train # dataset root dir
train: images  # train images (relative to 'path')
val: images  # val images (relative to 'path')


                           # Classes
names:
  0: Truck
  1: Person
  2: Tree""")

In [14]:
results = model.train(data='/content/config.yaml', epochs=10)

Ultralytics YOLOv8.0.149 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (AMD EPYC 7B12)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/config.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=F

FileNotFoundError: ignored

In [ ]:
!scp -r runs '/content/drive/MyDrive/Computer Vision Projects/Yolo/'

In [ ]:
metrics = model.val()  # evaluate model performance on the validation set

Ultralytics YOLOv8.0.147 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients
val: Scanning /content/data/labels.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:37<00:00,  5.35s/it]
                   all        100        161      0.576       0.36       0.43      0.237
Speed: 1.6ms preprocess, 284.0ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs/detect/val2


In [ ]:
results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
path = model.export(format="onnx")  # export the model to ONNX format

In [ ]:
!scp -r runs '/content/'

Pre trained

In [ ]:
# Load a model
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/Computer Vision Projects/Yolo/runs/detect/train5/weights/best.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="coco128.yaml", epochs=3)  # train the model
metrics = model.val()  # evaluate model performance on the validation set

Ultralytics YOLOv8.0.148 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/Computer Vision Projects/Yolo/runs/detect/train5/weights/best.pt, data=coco128.yaml, epochs=3, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchs

In [ ]:
results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
path = model.export(format="onnx")  # export the model to ONNX format


Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 /content/OIDv4_ToolKit/bus.jpg: 640x480 (no detections), 333.9ms
Speed: 5.7ms preprocess, 333.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)
Ultralytics YOLOv8.0.148 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.14.0 opset 17...
ONNX: export success ✅ 2.6s, saved as 'runs/detect/train/weights/best.onnx' (12.2 MB)

Export complete (4.7s)
Results saved to /content/OIDv4_ToolKit/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.onnx imgsz=640 
Validate:        yolo val task=detect model=runs/detect/train/weights/best.onnx imgsz=640 data=None 
Visualize:       https://netron.app


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="coco128.yaml", epochs=3)  # train the model
metrics = model.val()  # evaluate model performance on the validation set
results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
path = model.export(format="onnx")  # export the model to ONNX format

#Classification

Adjust dataset for the Classification task only

In [ ]:
labels = ['Tree', 'Person', 'Truck']

for label in labels:
  os.makedirs(f'/content/data/train/{label}', exist_ok=True)
  os.makedirs(f'/content/data/train/{label}', exist_ok=True)

  os.makedirs(f'/content/data/val/{label}', exist_ok=True)
  os.makedirs(f'/content/data/val/{label}', exist_ok=True)

In [ ]:
path = '/content/data/obj/train/'
DATA_ROOT = '/content/data/'

VAL_THRESHOLD = 0.2

for category in os.listdir(path):
  category_path = path+category

  for i in os.listdir(category_path):
    if i[-4:] == '.jpg':
      file_name = i
      image_file = category_path + '/' + i
      rand_num = np.random.rand()
      target = 'val' if rand_num <= VAL_THRESHOLD else 'train'
      os.rename(image_file, f'/content/data/{target}/{category}/{i}')

In [ ]:
os.rename('/content/data/obj', '/content/obj')
os.rename('/content/data/obj.names', '/content/obj.names')
os.rename('/content/data/train.txt', '/content/train.txt')

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n-cls.pt')

results = model.train(data='/content/data', epochs=20)

Ultralytics YOLOv8.0.148 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/data, epochs=20, patience=50, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simpli

In [ ]:
!scp -r runs '/content/drive/MyDrive/Computer Vision Projects/Yolo/'
!scp -r runs '/content/'

In [ ]:
metrics = model.val()
metrics.top1   # top1 accuracy

Ultralytics YOLOv8.0.148 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
               classes   top1_acc   top5_acc: 100%|██████████| 4/4 [00:03<00:00,  1.09it/s]
                   all        0.8          1
Speed: 0.0ms preprocess, 16.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/val2


0.800000011920929

In [ ]:
metrics.top5   # top5 accuracy

1.0

In [ ]:
results = model('/content/blue-truck-homepage-618x340.jpg')  # predict on an image


image 1/1 /content/blue-truck-homepage-618x340.jpg: 224x224 Truck 1.00, Person 0.00, Tree 0.00, 30.6ms
Speed: 1.8ms preprocess, 30.6ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)


In [ ]:
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: None
 keypoints: None
 keys: ['probs']
 masks: None
 names: {0: 'Person', 1: 'Tree', 2: 'Truck'}
 orig_img: array([[[213, 172, 140],
         [211, 170, 138],
         [208, 167, 135],
         ...,
         [223, 203, 186],
         [225, 206, 191],
         [228, 209, 194]],
 
        [[213, 172, 140],
         [210, 169, 137],
         [207, 166, 134],
         ...,
         [225, 205, 188],
         [227, 208, 193],
         [229, 210, 195]],
 
        [[211, 170, 138],
         [209, 168, 136],
         [206, 165, 133],
         ...,
         [229, 209, 192],
         [228, 209, 194],
         [231, 212, 197]],
 
        ...,
 
        [[186, 197, 205],
         [181, 192, 200],
         [179, 190, 198],
         ...,
         [200, 209, 212],
         [198, 207, 210],
         [200, 209, 212]],
 
        [[186, 197, 205],
         [181, 192, 200],
         [180, 191, 199],
         ...,
         [198, 207, 210]

In [ ]:
import matplotlib.pyplot as plt

res = plt.imread('/content/blue-truck-homepage-618x340.jpg')
plt.imshow(res)